In [48]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from catboost import CatBoostRegressor


In [49]:

# Define the function to convert classification predictions to regression
# def process_prediction(preds):
    # final_preds = []
    # for pred in preds:
        # argmax = np.argmax(pred, axis=0)
        # if argmax == 0:
            # final_preds.append(-1 * pred[0])
        # elif argmax == 1:
            # final_preds.append(0)
        # else:
            # final_preds.append(pred[2])
    # return final_preds

In [50]:

def process_prediction(pred):
    final_pred = np.zeros_like(pred)  # Initialize final predictions array with zeros

    # Apply comparisons element-wise
    final_pred[pred < -0.5] = -1.0
    final_pred[(pred >= -0.5) & (pred <= 0.5)] = 0.0
    final_pred[pred > 0.5] = 1.0

    return final_pred

In [51]:

# Load the data
train = pd.read_csv('Train.csv').dropna(axis=0)  # Read in train, ignoring one row with missing data
test = pd.read_csv('Test.csv').fillna('')  # Read in test

In [52]:
train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [53]:
# Split data into training and validation sets
df_train, df_valid = train_test_split(train, test_size=0.2, random_state=42)

In [54]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

In [55]:
# Transform text data into TF-IDF features
X_train = vectorizer.fit_transform(df_train['safe_text'])
X_valid = vectorizer.transform(df_valid['safe_text'])
X_test = vectorizer.transform(test['safe_text'])

In [56]:
# Define labels
y_train = df_train['label']
y_valid = df_valid['label']

In [57]:
kf = KFold(n_splits=10, shuffle=True, random_state=2)

errlgb = []  # Initialize list to store RMSE values
y_pred_totlgb = []  # Initialize list to store total predictions

for fold_, (train_idx, valid_idx) in enumerate(kf.split(X_train)):
    X_train_fold, X_valid_fold = X_train[train_idx], X_train[valid_idx]
    y_train_fold, y_valid_fold = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    # Model
    model = CatBoostRegressor(
                            iterations=2000,
                            learning_rate=0.05,
                            depth=8,
                            eval_metric='RMSE',  # Changed to accuracy metric
                            random_seed=42,
                            bagging_temperature=0.2,
                            od_type='Iter',
                            metric_period=50,
                            od_wait=300
                            )
                
    model.fit(X_train_fold, y_train_fold,
                            use_best_model=True,
                            eval_set=(X_valid_fold, y_valid_fold),  # Provide validation set for early stopping
                            verbose=50)
    
    y_pred_prob = model.predict(X_test)
    y_pred = process_prediction(y_pred_prob)  # Convert probabilities to regression
    
    rmse = np.sqrt(mean_squared_error(y_valid_fold, process_prediction(model.predict(X_valid_fold))))
    print("RMSE: ", rmse)
    errlgb.append(rmse)
    y_pred_totlgb.append(y_pred)

0:	learn: 0.6403097	test: 0.6188039	best: 0.6188039 (0)	total: 316ms	remaining: 10m 31s
50:	learn: 0.5851390	test: 0.5725347	best: 0.5725347 (50)	total: 15.8s	remaining: 10m 5s
100:	learn: 0.5675480	test: 0.5630017	best: 0.5630017 (100)	total: 31.5s	remaining: 9m 52s
150:	learn: 0.5555657	test: 0.5578833	best: 0.5578833 (150)	total: 45.6s	remaining: 9m 17s
200:	learn: 0.5437872	test: 0.5530506	best: 0.5530417 (198)	total: 59.2s	remaining: 8m 49s
250:	learn: 0.5320892	test: 0.5494654	best: 0.5493712 (247)	total: 1m 13s	remaining: 8m 29s
300:	learn: 0.5219885	test: 0.5466888	best: 0.5466888 (300)	total: 1m 27s	remaining: 8m 12s
350:	learn: 0.5110175	test: 0.5434026	best: 0.5434026 (350)	total: 1m 42s	remaining: 7m 59s
400:	learn: 0.5003671	test: 0.5411538	best: 0.5411538 (400)	total: 1m 55s	remaining: 7m 38s
450:	learn: 0.4902365	test: 0.5385159	best: 0.5385159 (450)	total: 2m 7s	remaining: 7m 17s
500:	learn: 0.4811174	test: 0.5360682	best: 0.5360259 (498)	total: 2m 19s	remaining: 6m 57s

0:	learn: 0.6395805	test: 0.6297257	best: 0.6297257 (0)	total: 295ms	remaining: 9m 48s
50:	learn: 0.5846938	test: 0.5813431	best: 0.5813431 (50)	total: 12.6s	remaining: 7m 59s
100:	learn: 0.5664040	test: 0.5705033	best: 0.5705033 (100)	total: 24.4s	remaining: 7m 39s
150:	learn: 0.5539205	test: 0.5635839	best: 0.5635839 (150)	total: 36.4s	remaining: 7m 25s
200:	learn: 0.5417411	test: 0.5598489	best: 0.5598489 (200)	total: 48.2s	remaining: 7m 11s
250:	learn: 0.5299012	test: 0.5583626	best: 0.5582518 (244)	total: 59.8s	remaining: 6m 56s
300:	learn: 0.5165589	test: 0.5559895	best: 0.5559895 (300)	total: 1m 11s	remaining: 6m 43s
350:	learn: 0.5059094	test: 0.5543068	best: 0.5543068 (350)	total: 1m 23s	remaining: 6m 32s
400:	learn: 0.4960540	test: 0.5530604	best: 0.5530604 (400)	total: 1m 35s	remaining: 6m 19s
450:	learn: 0.4874346	test: 0.5520909	best: 0.5519303 (445)	total: 1m 47s	remaining: 6m 7s
500:	learn: 0.4796007	test: 0.5513222	best: 0.5512453 (490)	total: 1m 58s	remaining: 5m 55s
5

0:	learn: 0.6372620	test: 0.6495056	best: 0.6495056 (0)	total: 288ms	remaining: 9m 36s
50:	learn: 0.5822019	test: 0.6061825	best: 0.6061648 (48)	total: 12.5s	remaining: 7m 59s
100:	learn: 0.5633166	test: 0.5956289	best: 0.5956289 (100)	total: 24.5s	remaining: 7m 40s
150:	learn: 0.5507680	test: 0.5895709	best: 0.5895709 (150)	total: 36.7s	remaining: 7m 29s
200:	learn: 0.5391944	test: 0.5858989	best: 0.5858989 (200)	total: 48.4s	remaining: 7m 13s
250:	learn: 0.5263227	test: 0.5832049	best: 0.5831002 (249)	total: 1m	remaining: 6m 58s
300:	learn: 0.5145023	test: 0.5814886	best: 0.5814886 (300)	total: 1m 11s	remaining: 6m 44s
350:	learn: 0.5034816	test: 0.5802168	best: 0.5801318 (346)	total: 1m 23s	remaining: 6m 32s
400:	learn: 0.4935087	test: 0.5781362	best: 0.5781362 (400)	total: 1m 35s	remaining: 6m 19s
450:	learn: 0.4838421	test: 0.5764500	best: 0.5763466 (447)	total: 1m 46s	remaining: 6m 6s
500:	learn: 0.4751438	test: 0.5760650	best: 0.5759118 (494)	total: 1m 58s	remaining: 5m 54s
550:

0:	learn: 0.6371374	test: 0.6487715	best: 0.6487715 (0)	total: 275ms	remaining: 9m 9s
50:	learn: 0.5808957	test: 0.6146599	best: 0.6146599 (50)	total: 12.2s	remaining: 7m 46s
100:	learn: 0.5625627	test: 0.6059650	best: 0.6059650 (100)	total: 23.7s	remaining: 7m 24s
150:	learn: 0.5501611	test: 0.6002263	best: 0.6002263 (150)	total: 35.2s	remaining: 7m 10s
200:	learn: 0.5383234	test: 0.5960307	best: 0.5960307 (200)	total: 46.8s	remaining: 6m 58s
250:	learn: 0.5257411	test: 0.5927023	best: 0.5926512 (249)	total: 58.4s	remaining: 6m 46s
300:	learn: 0.5139643	test: 0.5900395	best: 0.5900395 (300)	total: 1m 10s	remaining: 6m 36s
350:	learn: 0.5024228	test: 0.5871783	best: 0.5871783 (350)	total: 1m 21s	remaining: 6m 24s
400:	learn: 0.4924094	test: 0.5857276	best: 0.5855972 (397)	total: 1m 33s	remaining: 6m 11s
450:	learn: 0.4842917	test: 0.5845556	best: 0.5845412 (449)	total: 1m 45s	remaining: 6m
500:	learn: 0.4754094	test: 0.5831576	best: 0.5831553 (499)	total: 1m 56s	remaining: 5m 48s
550:	

0:	learn: 0.6377586	test: 0.6456676	best: 0.6456676 (0)	total: 240ms	remaining: 8m
50:	learn: 0.5824184	test: 0.5976600	best: 0.5976600 (50)	total: 12s	remaining: 7m 37s
100:	learn: 0.5645979	test: 0.5867554	best: 0.5867554 (100)	total: 23.6s	remaining: 7m 24s
150:	learn: 0.5515191	test: 0.5808104	best: 0.5808104 (150)	total: 35.3s	remaining: 7m 12s
200:	learn: 0.5401807	test: 0.5766582	best: 0.5766582 (200)	total: 47.1s	remaining: 7m 1s
250:	learn: 0.5272509	test: 0.5720206	best: 0.5720206 (250)	total: 58.8s	remaining: 6m 49s
300:	learn: 0.5140194	test: 0.5678734	best: 0.5678325 (299)	total: 1m 10s	remaining: 6m 37s
350:	learn: 0.5031216	test: 0.5659470	best: 0.5659227 (349)	total: 1m 21s	remaining: 6m 24s
400:	learn: 0.4930782	test: 0.5638055	best: 0.5637550 (397)	total: 1m 33s	remaining: 6m 12s
450:	learn: 0.4835216	test: 0.5619708	best: 0.5619708 (450)	total: 1m 45s	remaining: 6m 2s
500:	learn: 0.4741212	test: 0.5613527	best: 0.5612533 (495)	total: 1m 56s	remaining: 5m 50s
550:	lea

0:	learn: 0.6376918	test: 0.6463628	best: 0.6463628 (0)	total: 274ms	remaining: 9m 7s
50:	learn: 0.5816273	test: 0.6040207	best: 0.6040207 (50)	total: 12.9s	remaining: 8m 12s
100:	learn: 0.5641915	test: 0.5941373	best: 0.5941373 (100)	total: 24.4s	remaining: 7m 39s
150:	learn: 0.5521254	test: 0.5883247	best: 0.5883247 (150)	total: 35.9s	remaining: 7m 20s
200:	learn: 0.5401075	test: 0.5835884	best: 0.5835860 (199)	total: 47.5s	remaining: 7m 4s
250:	learn: 0.5281402	test: 0.5809909	best: 0.5809020 (249)	total: 58.9s	remaining: 6m 50s
300:	learn: 0.5158953	test: 0.5785506	best: 0.5785506 (300)	total: 1m 10s	remaining: 6m 37s
350:	learn: 0.5051611	test: 0.5757764	best: 0.5757764 (350)	total: 1m 24s	remaining: 6m 37s
400:	learn: 0.4967038	test: 0.5752730	best: 0.5751132 (396)	total: 1m 37s	remaining: 6m 28s
450:	learn: 0.4865021	test: 0.5735149	best: 0.5734239 (447)	total: 1m 52s	remaining: 6m 26s
500:	learn: 0.4776866	test: 0.5722319	best: 0.5722319 (500)	total: 2m 6s	remaining: 6m 18s
550

0:	learn: 0.6396712	test: 0.6251641	best: 0.6251641 (0)	total: 288ms	remaining: 9m 35s
50:	learn: 0.5849935	test: 0.5804514	best: 0.5804514 (50)	total: 13.2s	remaining: 8m 24s
100:	learn: 0.5656254	test: 0.5725674	best: 0.5725446 (98)	total: 27.1s	remaining: 8m 30s
150:	learn: 0.5531009	test: 0.5676948	best: 0.5676948 (150)	total: 41.8s	remaining: 8m 32s
200:	learn: 0.5405164	test: 0.5640570	best: 0.5640570 (200)	total: 54.5s	remaining: 8m 7s
250:	learn: 0.5291629	test: 0.5627263	best: 0.5625286 (248)	total: 1m 7s	remaining: 7m 48s
300:	learn: 0.5172130	test: 0.5619971	best: 0.5617643 (291)	total: 1m 19s	remaining: 7m 31s
350:	learn: 0.5059785	test: 0.5614498	best: 0.5614103 (349)	total: 1m 32s	remaining: 7m 16s
400:	learn: 0.4954286	test: 0.5605705	best: 0.5603871 (393)	total: 1m 45s	remaining: 6m 59s
450:	learn: 0.4862999	test: 0.5596259	best: 0.5595215 (446)	total: 1m 57s	remaining: 6m 43s
500:	learn: 0.4787787	test: 0.5583225	best: 0.5582878 (494)	total: 2m 9s	remaining: 6m 28s
550

0:	learn: 0.6365753	test: 0.6588494	best: 0.6588494 (0)	total: 312ms	remaining: 10m 24s
50:	learn: 0.5821284	test: 0.6093656	best: 0.6093656 (50)	total: 12.6s	remaining: 8m
100:	learn: 0.5640561	test: 0.5992150	best: 0.5992150 (100)	total: 25s	remaining: 7m 49s
150:	learn: 0.5509973	test: 0.5929619	best: 0.5929619 (150)	total: 37.4s	remaining: 7m 38s
200:	learn: 0.5402193	test: 0.5902728	best: 0.5901912 (199)	total: 49.9s	remaining: 7m 26s
250:	learn: 0.5279907	test: 0.5874644	best: 0.5873994 (247)	total: 1m 2s	remaining: 7m 13s
300:	learn: 0.5157700	test: 0.5840882	best: 0.5839728 (297)	total: 1m 14s	remaining: 7m
350:	learn: 0.5036606	test: 0.5821472	best: 0.5821472 (350)	total: 1m 27s	remaining: 6m 48s
400:	learn: 0.4943129	test: 0.5808942	best: 0.5807951 (397)	total: 1m 39s	remaining: 6m 35s
450:	learn: 0.4843745	test: 0.5800301	best: 0.5800301 (450)	total: 1m 51s	remaining: 6m 23s
500:	learn: 0.4768905	test: 0.5788754	best: 0.5788754 (500)	total: 2m 3s	remaining: 6m 10s
550:	learn

0:	learn: 0.6399411	test: 0.6250675	best: 0.6250675 (0)	total: 304ms	remaining: 10m 7s
50:	learn: 0.5850127	test: 0.5782845	best: 0.5782845 (50)	total: 12.8s	remaining: 8m 9s
100:	learn: 0.5670878	test: 0.5659404	best: 0.5659404 (100)	total: 25.1s	remaining: 7m 52s
150:	learn: 0.5543060	test: 0.5592483	best: 0.5592483 (150)	total: 37.4s	remaining: 7m 37s
200:	learn: 0.5422542	test: 0.5542749	best: 0.5542749 (200)	total: 49.7s	remaining: 7m 24s
250:	learn: 0.5309054	test: 0.5517532	best: 0.5517532 (250)	total: 1m 2s	remaining: 7m 15s
300:	learn: 0.5170263	test: 0.5481153	best: 0.5481153 (300)	total: 1m 14s	remaining: 7m 2s
350:	learn: 0.5055100	test: 0.5456385	best: 0.5456385 (350)	total: 1m 27s	remaining: 6m 49s
400:	learn: 0.4953448	test: 0.5450525	best: 0.5448808 (396)	total: 1m 39s	remaining: 6m 36s
450:	learn: 0.4866553	test: 0.5445323	best: 0.5441455 (431)	total: 1m 51s	remaining: 6m 24s
500:	learn: 0.4776835	test: 0.5432819	best: 0.5432819 (500)	total: 2m 5s	remaining: 6m 16s
550

0:	learn: 0.6385205	test: 0.6390563	best: 0.6390563 (0)	total: 342ms	remaining: 11m 23s
50:	learn: 0.5813701	test: 0.5988206	best: 0.5988206 (50)	total: 13s	remaining: 8m 17s
100:	learn: 0.5620292	test: 0.5890615	best: 0.5890615 (100)	total: 25.8s	remaining: 8m 4s
150:	learn: 0.5481174	test: 0.5850531	best: 0.5850531 (150)	total: 38.4s	remaining: 7m 50s
200:	learn: 0.5365399	test: 0.5823641	best: 0.5823330 (199)	total: 50.9s	remaining: 7m 35s
250:	learn: 0.5243856	test: 0.5816641	best: 0.5816641 (250)	total: 1m 3s	remaining: 7m 22s
300:	learn: 0.5123268	test: 0.5806566	best: 0.5803945 (287)	total: 1m 15s	remaining: 7m 8s
350:	learn: 0.5017566	test: 0.5797491	best: 0.5795675 (344)	total: 1m 28s	remaining: 6m 55s
400:	learn: 0.4912837	test: 0.5795321	best: 0.5794556 (399)	total: 1m 40s	remaining: 6m 42s
450:	learn: 0.4820827	test: 0.5797743	best: 0.5792852 (414)	total: 1m 53s	remaining: 6m 29s
500:	learn: 0.4736362	test: 0.5792863	best: 0.5790915 (493)	total: 2m 6s	remaining: 6m 17s
550:

In [ ]:
# Perform grid search

# Define evaluation set
eval_set = [(X_valid_fold, y_valid_fold)]

# Define parameter grid for grid search
param_grid = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [4, 6],
    'l2_leaf_reg': [1, 3]
}

# Initialize CatBoost classifier
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3,
                        scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train_fold, y_train_fold, eval_set=eval_set)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


0:	learn: 0.6394934	test: 0.6342374	best: 0.6342374 (0)	total: 52.7ms	remaining: 5.21s
50:	learn: 0.6224671	test: 0.6183970	best: 0.6183970 (50)	total: 2.39s	remaining: 2.29s
99:	learn: 0.6129347	test: 0.6101908	best: 0.6101908 (99)	total: 4.35s	remaining: 0us

bestTest = 0.6101907655
bestIteration = 99

[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   5.1s


0:	learn: 0.6449197	test: 0.6340773	best: 0.6340773 (0)	total: 40.9ms	remaining: 4.04s
50:	learn: 0.6255606	test: 0.6186695	best: 0.6186695 (50)	total: 2.17s	remaining: 2.08s
99:	learn: 0.6153944	test: 0.6110613	best: 0.6110613 (99)	total: 4.42s	remaining: 0us

bestTest = 0.6110613091
bestIteration = 99

[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   5.1s


0:	learn: 0.6432136	test: 0.6340088	best: 0.6340088 (0)	total: 42ms	remaining: 4.16s
50:	learn: 0.6239421	test: 0.6174370	best: 0.6174370 (50)	total: 2.7s	remaining: 2.59s
99:	learn: 0.6132115	test: 0.6085693	best: 0.6085693 (99)	total: 5.29s	remaining: 0us

bestTest = 0.6085693071
bestIteration = 99

[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   6.0s


0:	learn: 0.6356579	test: 0.6306666	best: 0.6306666 (0)	total: 38.3ms	remaining: 3.8s
50:	learn: 0.5805366	test: 0.5896111	best: 0.5896111 (50)	total: 2.29s	remaining: 2.2s
99:	learn: 0.5553899	test: 0.5794989	best: 0.5794989 (99)	total: 4.1s	remaining: 0us

bestTest = 0.5794988816
bestIteration = 99

[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.1; total time=   4.8s


...

0:	learn: 0.6346969	test: 0.6298012	best: 0.6298012 (0)	total: 147ms	remaining: 29.2s
50:	learn: 0.5728010	test: 0.5856578	best: 0.5856578 (50)	total: 4.96s	remaining: 14.5s
100:	learn: 0.5450417	test: 0.5789994	best: 0.5789405 (98)	total: 9.86s	remaining: 9.66s
150:	learn: 0.5172635	test: 0.5744695	best: 0.5744075 (148)	total: 15.8s	remaining: 5.13s
199:	learn: 0.4961607	test: 0.5728870	best: 0.5728669 (198)	total: 21.1s	remaining: 0us

bestTest = 0.5728668666
bestIteration = 198

Shrink model to first 199 iterations.
[CV] END depth=6, iterations=200, l2_leaf_reg=3, learning_rate=0.1; total time=  22.2s


0:	learn: 0.6390942	test: 0.6295200	best: 0.6295200 (0)	total: 223ms	remaining: 44.5s
50:	learn: 0.5754240	test: 0.5879809	best: 0.5879809 (50)	total: 7.13s	remaining: 20.8s
100:	learn: 0.5483202	test: 0.5783452	best: 0.5783452 (100)	total: 13.8s	remaining: 13.6s
150:	learn: 0.5224054	test: 0.5731711	best: 0.5731711 (150)	total: 19.6s	remaining: 6.35s
199:	learn: 0.5019054	test: 0.5699438	best: 0.5697749 (198)	total: 24.5s	remaining: 0us

bestTest = 0.5697748746
bestIteration = 198

Shrink model to first 199 iterations.
[CV] END depth=6, iterations=200, l2_leaf_reg=3, learning_rate=0.1; total time=  25.5s


0:	learn: 0.6376646	test: 0.6292131	best: 0.6292131 (0)	total: 117ms	remaining: 23.3s
50:	learn: 0.5704992	test: 0.5840000	best: 0.5840000 (50)	total: 6.35s	remaining: 18.5s
100:	learn: 0.5432552	test: 0.5769515	best: 0.5768468 (95)	total: 14.5s	remaining: 14.2s
150:	learn: 0.5179498	test: 0.5740229	best: 0.5740229 (150)	total: 20.6s	remaining: 6.7s
199:	learn: 0.4957669	test: 0.5713416	best: 0.5710796 (192)	total: 26.2s	remaining: 0us

bestTest = 0.5710796011
bestIteration = 192

Shrink model to first 193 iterations.
[CV] END depth=6, iterations=200, l2_leaf_reg=3, learning_rate=0.1; total time=  27.2s


0:	learn: 0.6371154	test: 0.6290151	best: 0.6290151 (0)	total: 382ms	remaining: 1m 15s
50:	learn: 0.5714496	test: 0.5817854	best: 0.5817854 (50)	total: 7.74s	remaining: 22.6s
100:	learn: 0.5451419	test: 0.5732161	best: 0.5732161 (100)	total: 15s	remaining: 14.7s
150:	learn: 0.5174120	test: 0.5681822	best: 0.5681281 (147)	total: 22.4s	remaining: 7.27s
199:	learn: 0.4983738	test: 0.5658776	best: 0.5656955 (191)	total: 29.2s	remaining: 0us

bestTest = 0.5656955336
bestIteration = 191

Shrink model to first 192 iterations.


GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x0000018046F23430>,
             param_grid={'depth': [4, 6], 'iterations': [100, 200],
                         'l2_leaf_reg': [1, 3], 'learning_rate': [0.01, 0.1]},
             scoring='neg_mean_squared_error', verbose=2)

In [ ]:
#  Print best parameters and corresponding RMSE for each fold
print(f"Fold {fold_ + 1} Best Parameters:", grid_search.best_params_)
print(f"Fold {fold_ + 1} Best RMSE:", np.sqrt(-grid_search.best_score_))


Fold 5 Best Parameters: {'depth': 6, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
Fold 5 Best RMSE: 0.5750940928968116


In [ ]:
# Use best model for predictions
best_model = grid_search.best_estimator_
valid_preds_catboost = best_model.predict(X_valid_fold)

In [ ]:
# Calculate RMSE on the validation set for each fold
rmse_catboost = np.sqrt(mean_squared_error(y_valid_fold, valid_preds_catboost))
print(f"Fold {fold_ + 1} Validation RMSE (CatBoost):", rmse_catboost)


Fold 5 Validation RMSE (CatBoost): 0.5656955336155786


In [ ]:
# Make predictions on the test set for each fold
test_preds_catboost = best_model.predict(X_test)

In [ ]:
# Make a submission dataframe for CatBoost for each fold (optional)
sub_catboost = pd.DataFrame({
        'tweet_id': test['tweet_id'],
        'label': test_preds_catboost.reshape(-1)
    })
sub_catboost.to_csv(f'catboost_submission_gridsearch_fold_.csv', index=False)
sub_catboost.head()


,tweet_id,label
0,00BHHHP1,-0.010336
1,00UNMD0E,0.438846
2,01AXPTJF,0.342040
3,01HOEQJW,0.483208
4,01JUKMAO,0.213619
